##1. 데이터 읽기

In [4]:
import pandas as pd
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
df = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
df['label'] = cancer.target
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,label
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


#2. 모델 만들기
> voting_classifier(분류기 리스트, voting="hard, soft")

##2.1 데이터 분류

In [ ]:
df_data = df.iloc[0:, :-1]
df_label = df.iloc[0:,-1]

> -1은 index의 마지막 위치에 class feature를 추가 하였음으로 마지막 이전 데이터와 마지막 데티어만 추출하여 data와 labels로 사용할 수 있다. 의미함으로 df_data는 마지막 전까지의 데이터를 얻어오게 되는 것이며 df_labels는 마지막 값만 얻어 오게 되는 것이다.

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_data, df_label, test_size=0.3, random_state=62)
y_train[:10]

483    1
228    1
511    1
411    1
308    1
357    1
417    0
493    1
296    1
235    1
Name: label, dtype: int64

##2.2 모델 만들기

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

lr = LogisticRegression(max_iter=5000)
knn = KNeighborsClassifier()
vo = VotingClassifier([('lr_id', lr), ('knn_id', knn)], voting='soft')

> 로지스틱 회귀와 KNN 두 가지 알고리즘을 이용하여 보팅을 진행할 것이다. 

> VotingClassifier의 첫 인자는 알고리즘 리스트가 들어 간다. 현재 ('lr_id', lr) 이와 같은 형식으로 들어 갔는데, 이 부분은 추후 하이퍼 파라미터 적용 시 로지스틱 회귀와 KNN에서 사용되는 파아미터가 다르기 떄문에 이를 식별하는 식별자로 lr_id를 만들어 둔 것이다.

##2.3 학습 및 예측

In [8]:
from sklearn.metrics import accuracy_score

vo.fit(X_train, y_train)
pred = vo.predict(X_test)

accuracy_score(y_test, pred)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9122807017543859

##2.4 모델별 성능 평가

In [21]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

lr = LogisticRegression(max_iter=5000)
knn = KNeighborsClassifier()
vo = VotingClassifier([('lr_id', lr), ('knn_id', knn)], voting='soft')

vo.fit(X_train, y_train)
pred = vo.predict(X_test)

print('voting 정확도 : ', accuracy_score(y_test, pred), '\n')

for clf in [lr, knn]:
  print('클래스 정보 : ', clf.__class__)
  clf_name = clf.__class__.__name__
  clf.fit(X_train, y_train)
  pred = clf.predict(X_test)
  print(clf_name, '의 정확도 :', accuracy_score(pred, y_test))

print('\n====================================================================\n')
#표준화
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)

vo = VotingClassifier([('lr_id', lr), ('knn_id', knn)], voting='soft')

vo.fit(X_train, y_train)
pred = vo.predict(X_test)

print('voting 정확도 : ', accuracy_score(y_test, pred), '\n')
for clf in [lr, knn]:
  clf_name = clf.__class__.__name__
  clf.fit(X_train, y_train)
  pred = clf.predict(X_test)
  print(clf_name, '의 정확도 :', accuracy_score(pred, y_test))

voting 정확도 :  0.9473684210526315 

클래스 정보 :  <class 'sklearn.linear_model._logistic.LogisticRegression'>
LogisticRegression 의 정확도 : 0.9590643274853801
클래스 정보 :  <class 'sklearn.neighbors._classification.KNeighborsClassifier'>
KNeighborsClassifier 의 정확도 : 0.935672514619883


voting 정확도 :  0.9473684210526315 

LogisticRegression 의 정확도 : 0.9590643274853801
KNeighborsClassifier 의 정확도 : 0.935672514619883


#3. 스케일링을 통한 모델 성능 높히기
> 로지스틱 회귀의 경우 데이터를 스케일링하게 되면 성능이 향상됨으로 StandardScaler를 이용할 것이다.

In [32]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
standard_scaler = scaler.fit_transform(df_data)


In [41]:
X_train, X_test, y_train, y_test = train_test_split(standard_scaler, df_label.values.reshape(-1), test_size=0.3, random_state=62)

for clf in [lr, knn]:
  print('클래스 정보 : ', clf.__class__)
  clf_name = clf.__class__.__name__
  clf.fit(X_train, y_train)
  pred = clf.predict(X_test)
  print(clf_name, '의 정확도 :', accuracy_score(pred, y_test))

클래스 정보 :  <class 'sklearn.linear_model._logistic.LogisticRegression'>
LogisticRegression 의 정확도 : 0.9649122807017544
클래스 정보 :  <class 'sklearn.neighbors._classification.KNeighborsClassifier'>
KNeighborsClassifier 의 정확도 : 0.9415204678362573


#4. 교차 검증

In [42]:
import numpy as np
from sklearn.model_selection import cross_val_score

for clf in [vo, lr, knn]:
  scores = cross_val_score(clf, np.concatenate((X_train, X_test)), np.concatenate((y_train, y_test)), scoring='accuracy', cv=5)
  print(clf.__class__.__name__)
  print(scores)
  print(scores.mean())
  print('======================================================================================================================')

VotingClassifier
[0.98245614 0.99122807 0.96491228 0.97368421 0.96460177]
0.9753764943331781
LogisticRegression
[0.95614035 0.99122807 0.97368421 0.97368421 0.95575221]
0.9700978108989288
KNeighborsClassifier
[0.97368421 0.99122807 0.98245614 0.93859649 0.92920354]
0.9630336904207422


> np.concatenate는 두 데이터를 합하는 함수이다.

#5. 하이퍼 파라미터

In [43]:
from sklearn.model_selection import GridSearchCV

def get_best_hyper_paramerter(logreg_list, n_neighbors_list):
  params =  {
      # 위에서 지정한 이름(lr_id, knn_id) + '__' + '파라미터' 형식으로 파라미터를 주게 된다.
      # voting 생성 시 사용한 이름 + '__' + 파라미터
    'lr_id__C' : logreg_list,
    'knn_id__n_neighbors' : n_neighbors_list,
    'knn_id__weights' : ["uniform", "distance"],
    'knn_id__metric' : ['euclidean', 'manhattan', 'minkowski']
  }

  grid_cv = GridSearchCV(vo, param_grid=params, scoring='accuracy', cv=5)
  grid_cv.fit(X_train, y_train)
  print('Best Score: ', grid_cv.best_score_)
  print('Best Hyper Parameter: ', grid_cv.best_params_)

In [44]:
logreg_list = [0.001, 0.01, 0.1, 1, 10, 25, 50, 100]
n_neighbors_list = list(range(1, 100, 10))
get_best_hyper_paramerter(logreg_list, n_neighbors_list)

Best Score:  0.9824050632911392
Best Hyper Parameter:  {'knn_id__metric': 'manhattan', 'knn_id__n_neighbors': 11, 'knn_id__weights': 'uniform', 'lr_id__C': 10}


> Best Score:  0.9824050632911392

> Best Hyper Parameter:  
- 'knn_id__metric': 'euclidean'
- 'knn_id__n_neighbors': 11
- 'knn_id__weights': 'distance'
- 'lr_id__C': 10}

In [45]:
logreg_list = list(range(1, 25, 5))
n_neighbors_list = list(range(1, 22, 5))
get_best_hyper_paramerter(logreg_list, n_neighbors_list)

Best Score:  0.9849367088607595
Best Hyper Parameter:  {'knn_id__metric': 'euclidean', 'knn_id__n_neighbors': 6, 'knn_id__weights': 'uniform', 'lr_id__C': 1}


In [49]:
logreg_list = list(range(1, 15, 1))
n_neighbors_list = list(range(3, 22, 1))
get_best_hyper_paramerter(logreg_list, n_neighbors_list)

Best Score:  0.9849683544303798
Best Hyper Parameter:  {'knn_id__metric': 'manhattan', 'knn_id__n_neighbors': 8, 'knn_id__weights': 'uniform', 'lr_id__C': 1}


# 최적의 파라미터

In [51]:
X_train, X_test, y_train, y_test = train_test_split(df_data, df_labels, test_size=0.3, random_state=62)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)

lr = LogisticRegression(max_iter=5000, C=14)
knn = KNeighborsClassifier(metric='manhattan', n_neighbors=3, weights='uniform')
vo = VotingClassifier([('lr_id', lr), ('knn_id', knn)], voting='soft')

vo.fit(X_train, y_train)
pred = vo.predict(X_test)

print('voting 정확도 : ', accuracy_score(y_test, pred), '\n')
for clf in [lr, knn]:
  clf_name = clf.__class__.__name__
  clf.fit(X_train, y_train)
  pred = clf.predict(X_test)
  print(clf_name, '의 정확도 :', accuracy_score(pred, y_test))

voting 정확도 :  0.9532163742690059 

LogisticRegression 의 정확도 : 0.9649122807017544
KNeighborsClassifier 의 정확도 : 0.9239766081871345


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usi